In [1]:
import pandas as pd
import os
# from os import path
import re
from itertools import groupby
from operator import itemgetter


LIMIT = 100
PATH = "./input"
DIR_LIST = [i for i in os.listdir(PATH)]
print(DIR_LIST)


['confirmed-cases-since-120465.csv', 'confirmed-cases-since-120465.xlsx', 'DESCRIBE_LOG_EVENTS_2021_01_24_20210725_170200.csv', 'DESCRIBE_LOG_EVENTS_2023 0625_164902.txt', 'DESCRIBE_LOG_EVENTS_20230625_154002.txt', 'DESCRIBE_LOG_EVENTS_20230625_160411.txt', 'DESCRIBE_LOG_EVENTS_20230625_160416.txt', 'DESCRIBE_LOG_EVENTS_20230625_160558.txt', 'DESCRIBE_LoG_EVENTS_20230625_165129.txt', 'DESCRIBE_LOG_EVENTS_20230625_254726.txt', 'ha_aod_003_dataenv_2.csv']


In [2]:
accumulated_log = []
has_added_header = False
_temp_filepaths = []
# for file_name in [i for i in DIR_LIST if re.match("^describe_log_events_\d{8}_\d{6}.txt$", i)]:
for file_name in [i for i in DIR_LIST if re.match("^DESCRIBE_LOG_EVENTS_[0-9]{8}_[0-1]{1}[0-9]{1}[0-9]{4}.txt$", i)]:
    file_path = f"{PATH}/{file_name}"
    
    is_file = os.path.isfile(file_path)

    if is_file:
        _temp_filepaths.append(file_path)
        with open(file_path, 'r') as file_log:
            print(f"File: {file_path}")
            is_header = True
            # i = 0
            line_count = 0
            # print(file_log.read())
            for line in file_log:
                if (is_header):
                    if not (has_added_header):
                        accumulated_log.append(line.split('|'))
                        has_added_header = True
                    is_header = False
                else:
                    # i = i + 1
                    # if (i >= 3):
                    #     break
                    # if (line_count == LIMIT):
                    #     break

                    if not (re.match("^\d{4}.+$", line)):
                        continue
                    # print(line.split('|'))
                    # print(f"line count: {line_count}")
                    accumulated_log.append(line.split("|"))
                    line_count += 1


    # break


File: ./input/DESCRIBE_LOG_EVENTS_20230625_154002.txt
File: ./input/DESCRIBE_LOG_EVENTS_20230625_160416.txt
File: ./input/DESCRIBE_LOG_EVENTS_20230625_160558.txt


In [3]:
print(len(accumulated_log))


450001


In [4]:
list(enumerate(accumulated_log[0]))

[(0, 'DATE_TIME'),
 (1, 'NAME'),
 (2, 'CITY'),
 (3, 'ZIPCODE'),
 (4, 'BBAN'),
 (5, 'LOCALE'),
 (6, 'BANK_COUNTRY'),
 (7, 'IBAN'),
 (8, 'COUNTRY_CALLING_CODE'),
 (9, 'MSISDN'),
 (10, 'PHONE_NUMBER'),
 (11, 'STATUS'),
 (12, 'GENDER\n')]

In [5]:
def filter_active(record):
    if (record[11] == 'active'):
        return True
    else:
        return False
        
i = 0
active_logs = list(filter(filter_active, accumulated_log))
# for record in accumulated_log:
#     if (filter_active(record)):
#         active_log.append(record)
#         i += 1

# print(type(active_logs))
print(f"active records : {len(active_logs)}")
# for i in active_logs:
#     print(i)  

active records : 224553


In [6]:
group_by_gender_output = []
_temp_genders = []
for _, value in groupby(sorted(active_logs, key=itemgetter(12)), key=itemgetter(12)):
    value = list(value)
    value_count = len(value)
    print(f"gender: {value[0][12]} | count: {value_count}")
    _temp_genders.append(f"gender: {value[0][12]} | count: {value_count}")
    # break
    # print(value)
    group_by_gender_output.append(value)

gender: f
 | count: 112215
gender: m
 | count: 112338


In [7]:
sorted_by_zipcode = sorted(active_logs, key=itemgetter(3))
_temp_zipcode = f"minimum zipcode: {sorted_by_zipcode[0][3]}\nmaximum zipcode: {sorted_by_zipcode[-1][3]}"
print(f"minimum zipcode: {sorted_by_zipcode[0][3]}\nmaximum zipcode: {sorted_by_zipcode[-1][3]}")

minimum zipcode: 00501
maximum zipcode: 99950


In [8]:
def filter_phonenumber_format(record):
    if (re.match("^\(\d{3}\)\d{3}-\d{4}$", record[10])):
        return True
    else:
        return False

phonenumber_output = []
phonnumber_formats = list(filter(filter_phonenumber_format, active_logs))
print(len(phonnumber_formats))
# for i in phonnumber_formats:
#     print(i[10])
with open("./special_phonenumber.txt", "w") as file:
    for i in phonnumber_formats:
        file.write(f'{i[10]}\n')
    # file.writelines(phonnumber_formats)

18130


In [9]:
print("Summary")

print(f"Processed files:")
for i in _temp_filepaths:
    print(i)
print(f"Total records: {len(accumulated_log)-1}")

print(f"status active records : {len(active_logs)}")

for i in _temp_genders:
    print(i)

print(_temp_zipcode)

answers_text = f"""

""" 

Summary
Processed files:
./input/DESCRIBE_LOG_EVENTS_20230625_154002.txt
./input/DESCRIBE_LOG_EVENTS_20230625_160416.txt
./input/DESCRIBE_LOG_EVENTS_20230625_160558.txt
Total records: 450000
status active records : 224553
gender: f
 | count: 112215
gender: m
 | count: 112338
minimum zipcode: 00501
maximum zipcode: 99950
